In [ ]:
import numpy as np
import struct
import math
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import matplotlib as mpl
import pegasus_read as pegr
from matplotlib.pyplot import *
from scipy.ndimage import gaussian_filter
from get_kparofkprp import get_kparofkprp
import xarray
import scipy.io
import h5py
import bottleneck as bn

In [ ]:
name = 'b_b3_sim1'
nspecies = 1#7
plotsp = [1] #[0,2,1]
#charge = [1,1,1]
#mass = [1,1,1]
charge = [1,5,2]
mass = [1,16,4]


#output range  phi   ()
it0 = 50#25#0 #25 #65
it1 = 76#50#76 #50 #144
#output range f
itf0 = 0#146#73#146#73#0
itf1 = 222#146#222#222
#25 and 73, 50 and 146
#Try using a later range to match better (and use what constants Silvio has for each of his runs)
interv = "50-76/"
#interv = "25-50/"
#interv = "25-76/"

# window avg of f
rolling_window = 15#15

#cooling corrections
it0corr = 0
it1corr = 23 #25
cooling_corr = False # shouldnt affect SH here? check... #False

In [ ]:
# box parameters (beta = 0.3)
aspct = 6
lprp = 48.1802/(2.0*np.pi)                # in (2*pi*d_i) units   divide by 2pi
lprl = lprp*aspct         # in (2*pi*d_i) units
Lperp = 2.0*np.pi*lprp    # in d_i units
Lpara = 2.0*np.pi*lprl    # in d_i units
N_perp = 280  #ncells in perp direction
N_para = N_perp*aspct     # assuming isotropic resolution
kperpdi0 = 1./lprp        # minimum k_perp ( = 2*pi / Lperp)
kparadi0 = 1./lprl        # minimum k_para ( = 2*pi / Lpara)
betai0 = 0.3 #1./9.            # ion plasma beta
tau0 = 1.                 # temperature ratio (Te/Ti)
beta0 = (1.+tau0)*betai0  # total plasma beta

#number of processors used   (check if he computes D locally on each proc or not)
# seems the f read in is a sum over all processors (not local)
n_proc = 350*56

#k_perp shells
nkshells = 198#200

#binning type
bin_type = "linear"

In [ ]:
#gaussian filter
apply_smoothing = True #False
smooth_method = 'gaussian'
filter_passes = 1 #20 #10
#gaussian filter
sigma_smoothing = 0.5 #0.33
#Savitzky-Golay filter
window_size = 5
polyn_order = 3

In [ ]:
#numpy gradient: derivative order at edges
edge_grad_order = 2

#nomalization methods
same_norm_all = False #True #False

#exponential correction in diffusion coeff
exp_corr = True #False
c0exp = 0.09 #beta=0.3 0.05 #0.05

#shaded area: +/- c_pm_std*sigma (sigma = standard dev.)
c_pm_std = 1.0 #0.5

#--parameters for final plot  k_perp w_perp / \Omega = kappa  (translation from k_perp to w_perp for a given particle)
kappaU01 = 1.1 #beta = 0.3 1.25 #1.2
kappaB01 = 1.1 #1.25 #1.2
kappaPHI01 = 1.1 #1.25 #1.2

In [ ]:
#complement kpara*dBperp. dont know what this is
dBprp_complement = False #True #False
dtheta_dir = '1p5deg/'
path_strct_fnct = '../strct_fnct/'+dtheta_dir
m_order = '2'
cbprp_sign = +1.

#2*pi coefficient            k_perp lambda =1 or k_perp lambda = 2pi. Chose the former
TWOPIcoeff = False #True

#non-linear corrections
NLcorrections = False

In [ ]:
#figure format
output_figure = False #True #False
fig_frmt = ".pdf"#".png"#".pdf"
width_2columns = 512.11743/72.2
width_1column = 245.26653/72.2

#verbosity
verb_diag = False
verb_read = False

In [ ]:
#--rho_i units and KAW eigenvector normalization for density spectrum
kperprhoi0 = np.sqrt(betai0)*kperpdi0
kpararhoi0 = np.sqrt(betai0)*kparadi0
normKAW = betai0*(1.+betai0)*(1. + 1./(1. + 1./betai0))
#--
#--alfven speed (v_th units)
vA01 = np.sqrt(1./betai0)
#--d_i scale (rho_th units)
kdi01 = np.sqrt(betai0)

#--heating normalization
Qnormalization01 = 0.75 #0.4

In [ ]:
#paths
problem = "minor_turb"
path_save = "../figures/"
base = "../saved-analysis/spectrum_dat/"+name+"/"+name
#path_read_lev = "../fig_data/"

In [ ]:
# plot parameters

#latex fonts
font = 9
mpl.rc('text', usetex=True)
mpl.rc('font', family = 'serif')
mpl.rcParams['xtick.labelsize']=font-1
mpl.rcParams['ytick.labelsize']=font-1
#mpl.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
mpl.rcParams['contour.negative_linestyle'] = 'solid'
plt.rcParams["font.weight"] = "normal"
plt.rcParams['xtick.top']=True
plt.rcParams['ytick.right']=True


#Hawley colormap
bit_rgb = np.linspace(0,1,256)
colors = [(0,0,127), (0,3,255), (0,255,255), (128,128,128), (255,255,0),(255,0,0),(135,0,0)]
positions = [0.0,0.166667,0.333333,0.5,0.666667,0.833333,1]
for iii in range(len(colors)):
 colors[iii] = (bit_rgb[colors[iii][0]],
                bit_rgb[colors[iii][1]],
                bit_rgb[colors[iii][2]])

cdict = {'red':[], 'green':[], 'blue':[]}
for pos, color in zip(positions, colors):
 cdict['red'].append((pos, color[0], color[0]))
 cdict['green'].append((pos, color[1], color[1]))
 cdict['blue'].append((pos, color[2], color[2]))

cmap = mpl.colors.LinearSegmentedColormap('my_colormap',cdict,256)

In [ ]:
print("\n [ reading initial condition ]")
vdf0, time, vprp0, vprl0 = pegr.readmat_vdf(name,0,1,nspecies,plotsp,edv=False,grid=True,verbose=True)

#first normalization by number of processors
for i in np.arange(nspecies):
    vdf0[i] = vdf0[i][0,:,:] / float(n_proc)

In [ ]:
if cooling_corr:
    #correcting for numerical cooling
    print("\n [ calculate cooling correction (vs w_perp) ]")
    # if not apply_smoothing - store in edotv_prp_smooth arrays etc instead
    # can just do everything in one line instead
    # calculate here to not overload storage once main vdfs are read in
    blank1, blank2, edotv_prp_, edotv_prl_, blank3, blank4 = pegr.readmat_vdf(name,it0corr,it1corr+1,nspecies,plotsp,edv=True,grid=True,verbose=verb_read)

    for sp in np.arange(nspecies):
        edotv_prl_[sp] = edotv_prl_[sp] / float(n_proc)
        edotv_prp_[sp] = edotv_prp_[sp] / float(n_proc)

        if apply_smoothing:
            for ifilt in range(filter_passes):
              edotv_prp_[sp] = gaussian_filter(edotv_prp_[sp],sigma=sigma_smoothing,axes=(1,2))
              edotv_prl_[sp] = gaussian_filter(edotv_prl_[sp],sigma=sigma_smoothing,axes=(1,2))

    edotv_prl_corr = []
    edotv_prp_corr = []
    edotv_prl_smooth_corr = []
    edotv_prp_smooth_corr = []

    for sp in np.arange(nspecies):
        edotv_prl_corr.append(np.mean(edotv_prl_[sp],axis=0,keepdims=True))
        edotv_prp_corr.append(np.mean(edotv_prp_[sp],axis=0,keepdims=True))
        if (not apply_smoothing):
            edotv_prl_smooth_corr.append(gaussian_filter(edotv_prl_[sp],sigma=sigma_smoothing,axes=(1,2)))
            edotv_prp_smooth_corr.append(gaussian_filter(edotv_prp_[sp],sigma=sigma_smoothing,axes=(1,2)))
            for ifilt in range(filter_passes-1):
                edotv_prl_smooth_corr[sp] = gaussian_filter(edotv_prl_smooth_corr[sp],sigma=sigma_smoothing,axes=(1,2))
                edotv_prp_smooth_corr[sp] = gaussian_filter(edotv_prp_smooth_corr[sp],sigma=sigma_smoothing,axes=(1,2))
            edotv_prl_smooth_corr[sp] = np.mean(edotv_prl_smooth_corr[sp],axis=0,keepdims=True)
            edotv_prp_smooth_corr[sp] = np.mean(edotv_prp_smooth_corr[sp],axis=0,keepdims=True)

    del blank1, blank2, edotv_prp_, edotv_prl_, blank3, blank4

In [ ]:
### HEATING VS W_PERP (beta = 0.3)
#
# -> reading simulation data, time averaging, and cooling corrections
# -> also: reading spectra of fluctuations, reducing to k_perp spectra, and time averaging
#
print("\n ### HEATING VS W_PERP ###")

for sp in np.arange(nspecies):

    # read in VDF, edotv file saved using jono's scripts
    vdf_, time, edotv_prp_, edotv_prl_, vprp, vprl = pegr.readmat_vdf(name,itf0,itf1+1,1,plotsp[sp],edv=True,grid=True,verbose=verb_read)

    time = time[itf0:itf1+1,:]

    vdf_avg = []
    edotv_prp_t = []
    edotv_prl_t = []
    edotv_prp_avg = []
    edotv_prl_avg = []
    dfdwprp_t = []
    dfdwprp_avg = []

    if verb_diag:
    print("\n")
    print("#########################################################")
    print("### v-space analysis: distribution function & heating ###")
    print("#########################################################")
    print("species",plotsp[sp])

    # is assumed dvprp and dvprl are the same for all species later on. (can change that)
    dvprp = vprp[sp][0,2,0]-vprp[sp][0,1,0]
    dvprl = vprl[sp][0,0,2]-vprl[sp][0,0,1]

    #first normalization by number of processors
    vdf_[sp] = vdf_[sp] / float(n_proc)
    edotv_prl_[sp] = edotv_prl_[sp] / float(n_proc)
    edotv_prp_[sp] = edotv_prp_[sp] / float(n_proc)

    if apply_smoothing:
    for ifilt in range(filter_passes):
      edotv_prp_[sp] = gaussian_filter(edotv_prp_[sp],sigma=sigma_smoothing,axes=(1,2))
      edotv_prl_[sp] = gaussian_filter(edotv_prl_[sp],sigma=sigma_smoothing,axes=(1,2))

    # average over a rolling window (could also try fixed bin avgs)
    vdf_avg.append(xarray.DataArray(vdf_[sp],dims=['t','wprp','wprl']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())
    edotv_prp_avg.append(xarray.DataArray(edotv_prp_[sp],dims=['t','wprp','wprl']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())
    edotv_prl_avg.append(xarray.DataArray(edotv_prl_[sp],dims=['t','wprp','wprl']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())

    edotv_prp_t.append(edotv_prp_[sp]*1.0)
    edotv_prl_t.append(edotv_prl_[sp]*1.0)

    #computing <df/dwprp>
    fvprp_t = np.sum(vdf_[sp]/vprp[sp]*dvprl,axis=2) / np.sum(np.sum(vdf_[sp]/vprp[sp]*dvprl*dvprl,axis=2),axis=1,keepdims=True)
    dfdwprp_t.append(np.gradient(fvprp_t,vprp[sp][0,:,0],edge_order=edge_grad_order,axis=1))
    dfdwprp_avg.append(xarray.DataArray(dfdwprp_t[sp],dims=['t','wprp']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())


    vdf_, time, edotv_prp_, edotv_prl_, vprp, vprl

In [ ]:
### HEATING VS W_PERP (beta = 0.3)
#
# -> reading simulation data, time averaging, and cooling corrections
# -> also: reading spectra of fluctuations, reducing to k_perp spectra, and time averaging
#
print("\n ### HEATING VS W_PERP ###")



# read in VDF, edotv file saved using jono's scripts
vdf_, time, edotv_prp_, edotv_prl_, vprp, vprl = pegr.readmat_vdf(name,itf0,itf1+1,nspecies,plotsp,edv=True,grid=True,verbose=verb_read)

time = time[itf0:itf1+1,:]

vdf_avg = []
edotv_prp_t = []
edotv_prl_t = []
edotv_prp_avg = []
edotv_prl_avg = []
dfdwprp_t = []
dfdwprp_avg = []

for sp in np.arange(nspecies):
    if verb_diag:
        print("\n")
        print("#########################################################")
        print("### v-space analysis: distribution function & heating ###")
        print("#########################################################")
        print("species",plotsp[sp])

    # is assumed dvprp and dvprl are the same for all species later on. (can change that)
    dvprp = vprp[sp][0,2,0]-vprp[sp][0,1,0]
    dvprl = vprl[sp][0,0,2]-vprl[sp][0,0,1]

    #first normalization by number of processors
    vdf_[sp] = vdf_[sp] / float(n_proc)
    edotv_prl_[sp] = edotv_prl_[sp] / float(n_proc)
    edotv_prp_[sp] = edotv_prp_[sp] / float(n_proc)

    if apply_smoothing:
        for ifilt in range(filter_passes):
          edotv_prp_[sp] = gaussian_filter(edotv_prp_[sp],sigma=sigma_smoothing,axes=(1,2))
          edotv_prl_[sp] = gaussian_filter(edotv_prl_[sp],sigma=sigma_smoothing,axes=(1,2))

    # average over a rolling window (could also try fixed bin avgs)
    vdf_avg.append(xarray.DataArray(vdf_[sp],dims=['t','wprp','wprl']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())
    edotv_prp_avg.append(xarray.DataArray(edotv_prp_[sp],dims=['t','wprp','wprl']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())
    edotv_prl_avg.append(xarray.DataArray(edotv_prl_[sp],dims=['t','wprp','wprl']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())

    edotv_prp_t.append(edotv_prp_[sp]*1.0)
    edotv_prl_t.append(edotv_prl_[sp]*1.0)

    #computing <df/dwprp>
    fvprp_t = np.sum(vdf_[sp]/vprp[sp]*dvprl,axis=2) / np.sum(np.sum(vdf_[sp]/vprp[sp]*dvprl*dvprl,axis=2),axis=1,keepdims=True)
    dfdwprp_t.append(np.gradient(fvprp_t,vprp[sp][0,:,0],edge_order=edge_grad_order,axis=1))
    dfdwprp_avg.append(xarray.DataArray(dfdwprp_t[sp],dims=['t','wprp']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())


    vdf_, time, edotv_prp_, edotv_prl_, vprp, vprl

In [ ]:
#vdf_[sp]/vprp[sp]
#vdf output is actually vperp*f: restoring f
# check if jono's script already corrects for this
vdf = []
for sp in np.arange(nspecies):
    vdf.append(vdf_avg[sp] / vprp[sp])
    vdf0[sp] = vdf0[sp] / vprp0[sp][0,:,:]

edotv_prl = edotv_prl_avg
edotv_prp = edotv_prp_avg
if (not apply_smoothing):
  edotv_prl_smooth = edotv_prl_smooth_avg
  edotv_prp_smooth = edotv_prp_smooth_avg

In [ ]:
print("\n")
print("########################## BETA = 0.3 ##########################")
print(" (from v space -- before cooling corrections) ")
print(" after 3.5t_A")
for sp in  np.arange(nspecies):
    print(" species ",plotsp[sp])
    print(" 1) integral of <Qperp> (code units): ",np.mean(np.sum(edotv_prp[sp][:,:,:]*dvprp*dvprl,axis=(1,2))))
    print(" 2) integral of <Qpar> (code units): ",np.mean(np.sum(edotv_prl[sp][:,:,:]*dvprp*dvprl,axis=(1,2))))
print("################################################################")
print("\n")

#computing d<f>/dw_perp
#dfdwprp = np.gradient(np.sum(vdf*dvprl,axis=1),vprp)
# f_vprp using rolling averaged f, and then calculating dfdwprp
f_vprp = []
f0_vprp = []
dfdwprp = []
dfdwprp_finitediff = []
gg_tmp = []
gg_tmp_smooth = []

for sp in np.arange(nspecies):
    f_vprp.append(np.sum(vdf[sp]*dvprl,axis=2)/np.abs(  np.sum(np.sum(vdf[sp]*dvprl*dvprp,axis=2),axis=1,keepdims=True)   ))
    f0_vprp.append(np.sum(vdf0[sp]*dvprl,axis=1)/np.abs(np.sum(vdf0[sp]*dvprl*dvprp)))
    dfdwprp.append(np.gradient(f_vprp[sp],vprp[sp][0,:,0],edge_order=edge_grad_order,axis=1))
    dfdwprp_finitediff.append(np.zeros(f_vprp[sp].shape))
    dfdwprp_finitediff[sp][:,1:-1] = (f_vprp[sp][:,2:]-f_vprp[sp][:,:-2])/(vprp[sp][:,2:,0]-vprp[sp][:,:-2,0])
    dfdwprp_finitediff[sp][:,0] = (f_vprp[sp][:,1]-f_vprp[sp][:,0])/(vprp[sp][:,1,0]-vprp[sp][:,0,0])
    dfdwprp_finitediff[sp][:,len(vprp[sp][0,:,0])-1] = (f_vprp[sp][:,len(vprp[sp][0,:,0])-1]-f_vprp[sp][:,len(vprp[sp][0,:,0])-2])/(vprp[sp][:,len(vprp[sp][0,:,0])-1,0]-vprp[sp][:,len(vprp[sp][0,:,0])-2,0])

# rolling avg edotv
    gg_tmp.append(edotv_prp[sp]/(np.abs( np.abs(np.sum(edotv_prl[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) + np.abs(np.sum(edotv_prp[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) )))
    edotv_prp_t[sp] /= (np.abs( np.abs(np.sum(edotv_prl_t[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) + np.abs(np.sum(edotv_prp_t[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) ))
    if (not apply_smoothing):
        gg_tmp_smooth.append(edotv_prp_smooth[sp]/(np.abs( np.abs(np.sum(edotv_prl_smooth[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) + np.abs(np.sum(edotv_prp_smooth[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) )))
        edotv_prp_smooth_t[sp] /= (np.abs( np.abs(np.sum(edotv_prl_smooth_t[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) + np.abs(np.sum(edotv_prp_smooth_t[sp]*dvprp*dvprl,axis=(1,2),keepdims=True)) ))

In [ ]:
if cooling_corr:
    #correcting for numerical cooling
    print("\n [ apply cooling correction (vs w_perp) ]")
    """
    # if not apply_smoothing - store in edotv_prp_smooth arrays etc instead
    # can just do everything in one line instead
    edotv_prl_corr.append(np.mean(edotv_prl_[sp][it0corr:it1corr+1,:,:],axis=0,keepdims=True))
    edotv_prp_corr.append(np.mean(edotv_prp_[sp][it0corr:it1corr+1,:,:],axis=0,keepdims=True))
    if (not apply_smoothing):
        edotv_prl_smooth_corr.append(gaussian_filter(edotv_prl_[sp][it0corr:it1corr+1,:,:],sigma=sigma_smoothing,axes=(1,2)))
        edotv_prp_smooth_corr.append(gaussian_filter(edotv_prp_[sp][it0corr:it1corr+1,:,:],sigma=sigma_smoothing,axes=(1,2)))
        for ifilt in range(filter_passes-1):
            edotv_prl_smooth_corr[sp] = gaussian_filter(edotv_prl_smooth_corr[sp],sigma=sigma_smoothing,axes=(1,2))
            edotv_prp_smooth_corr[sp] = gaussian_filter(edotv_prp_smooth_corr[sp],sigma=sigma_smoothing,axes=(1,2))
        edotv_prl_smooth_corr[sp] = np.mean(edotv_prl_smooth_corr[sp],axis=0,keepdims=True)
        edotv_prp_smooth_corr[sp] = np.mean(edotv_prp_smooth_corr[sp],axis=0,keepdims=True)
    """

    gg_tmp[sp] -= edotv_prp_corr[sp]/(np.abs( np.abs(np.sum(edotv_prl_corr[sp]*dvprp*dvprl,keepdims=True)) + np.abs(np.sum(edotv_prp_corr[sp]*dvprp*dvprl,keepdims=True)) ))
    edotv_prp_t[sp] -= edotv_prp_corr[sp]/(np.abs( np.abs(np.sum(edotv_prl_corr[sp]*dvprp*dvprl,keepdims=True)) + np.abs(np.sum(edotv_prp_corr[sp]*dvprp*dvprl,keepdims=True)) ))

    if (not apply_smoothing):
        gg_tmp_smooth[sp] -= edotv_prp_smooth_corr[sp]/(np.abs( np.abs(np.sum(edotv_prl_smooth_corr[sp]*dvprp*dvprl,keepdims=True)) + np.abs(np.sum(edotv_prp_smooth_corr[sp]*dvprp*dvprl,keepdims=True)) ))
        edotv_prp_smooth_t[sp] -= edotv_prp_smooth_corr[sp]/(np.abs( np.abs(np.sum(edotv_prl_smooth_corr[sp]*dvprp*dvprl,keepdims=True)) + np.abs(np.sum(edotv_prp_smooth_corr[sp]*dvprp*dvprl,keepdims=True)) ))

In [ ]:
Qprp_vprp = []
Qprp_vprp_t = []
Qprp_vprp_t_avg = []
Qprp_vprp_smooth = []
Qprp_vprp_smooth_t = []
Qprp_vprp_smooth_t_avg = []
stdQprp_vprp = []
stdQprp_vprp_smooth = []
Dprpprp = []
Dprpprp_sign = []
Dprpprp_t = []
Dprpprp_t_avg = []
Dprpprp_smooth = []
Dprpprp_sign_smooth = []
Dprpprp_smooth_t = []
Dprpprp_smooth_t_avg = []
stdDprpprp_vprp = []
stdDprpprp_vprp_smooth = []

for sp in np.arange(nspecies):
    # total perpendicular heating from edotv
    # does this need to be normalised by qom? since E field acceleration
    Qprp_vprp.append(np.sum(gg_tmp[sp]*dvprl,axis=2))
    Qprp_vprp_t.append(np.sum(edotv_prp_t[sp]*dvprl,axis=2))
    Qprp_vprp_t_avg.append(xarray.DataArray(Qprp_vprp_t[sp],dims=['t','wprp']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())
    if (not apply_smoothing):
      Qprp_vprp_smooth.append(np.sum(gg_tmp_smooth[sp]*dvprl,axis=2))
      Qprp_vprp_smooth_t.append(np.sum(edotv_prp_smooth_t[sp]*dvprl,axis=2))
      Qprp_vprp_smooth_t_avg.append(xarray.DataArray(Qprp_vprp_smooth_t[sp],dims=['t','wprp']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())

    #normalize Qprp_sim (after or before computing Dprpprp?)
    #normQ_sim[sp] = Qnormalization01/np.sum(np.abs(Qprp_vprp[sp])*dvprp,axis=1,keepdims=True)
    #print("  -> denom of normQ_sim = ",np.sum(np.abs(Qprp_vprp[sp])*dvprp,axis=1))
    Qprp_vprp[sp] *= Qnormalization01/np.sum(np.abs(Qprp_vprp[sp])*dvprp,axis=1,keepdims=True)
    #Qprp_vprp_t *= normQ_sim
    Qprp_vprp_t[sp] *= Qnormalization01/np.sum(np.abs(Qprp_vprp_t[sp])*dvprp,axis=1,keepdims=True)
    #for iit in range(it0,it1+1):
    #  Qprp_vprp_t[:,iit-it0] *= Qnormalization01/np.sum(np.abs(Qprp_vprp_t[:,iit-it0])*dvprp)
    #Qprp_vprp_t_avg *= normQ_sim
    Qprp_vprp_t_avg[sp] *= Qnormalization01/np.sum(np.abs(Qprp_vprp_t_avg[sp])*dvprp,axis=1,keepdims=True)

    #std in time
    stdQprp_vprp.append(np.std(Qprp_vprp_t[sp],axis=0))

    if (not apply_smoothing):
        Qprp_vprp_smooth[sp] *= Qnormalization01/np.sum(np.abs(Qprp_vprp_smooth[sp])*dvprp,axis=1,keepdims=True)
        Qprp_vprp_smooth_t[sp] *= Qnormalization01/np.sum(np.abs(Qprp_vprp_smooth_t[sp])*dvprp,axis=1,keepdims=True)
        Qprp_vprp_smooth_t_avg[sp] *= Qnormalization01/np.sum(np.abs(Qprp_vprp_smooth_t_avg[sp])*dvprp,axis=1,keepdims=True)
        stdQprp_vprp_smooth.append(np.std(Qprp_vprp_smooth_t[sp],axis=0))


    #computing diff coefficient  heating/dfdw
    #Get rid of divisions by zero that we get because of how many wprp we go out to (where there are no particles/f)
    #Q is mostly small in these regions...
    #dfdw0s = np.ones(dfdwprp[sp].shape)-np.isnan(1./dfdwprp[sp])
    #dfdw0s_t = np.ones(dfdwprp_t[sp].shape)-np.isnan(1./dfdwprp_t[sp])
    Dprpprp.append(- np.abs(Qprp_vprp[sp]) / dfdwprp[sp])
    Dprpprp[sp][np.isinf(1./dfdwprp[sp])]=0
    Dprpprp_sign.append(- Qprp_vprp[sp] / dfdwprp[sp])
    Dprpprp_sign[sp][np.isinf(1./dfdwprp[sp])]=0
    Dprpprp_t.append(- np.abs(Qprp_vprp_t[sp]) / dfdwprp_t[sp])
    Dprpprp_t[sp][np.isinf(1./dfdwprp_t[sp])]=0
    Dprpprp_t_avg.append(xarray.DataArray(Dprpprp_t[sp],dims=['t','wprp']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())
    #Dprpprp_t_avg[sp][np.isnan(Dprpprp_t_avg[sp])]=0
    stdDprpprp_vprp.append(np.std(Dprpprp_t[sp],axis=0))

    if (not apply_smoothing):
      Dprpprp_smooth.append(- np.abs(Qprp_vprp_smooth[sp]) / dfdwprp[sp])
      Dprpprp_smooth[sp][np.isinf(1./dfdwprp[sp])]=0
      Dprpprp_sign_smooth.append(- Qprp_vprp_smooth[sp] / dfdwprp[sp])
      Dprpprp_sign_smooth[sp][np.isinf(1./dfdwprp[sp])]=0
      Dprpprp_smooth_t.append(- np.abs(Qprp_vprp_smooth_t[sp]) / dfdwprp_t[sp])
      Dprpprp_smooth_t[sp][np.isinf(1./dfdwprp_t[sp])]=0
      Dprpprp_smooth_t_avg.append(xarray.DataArray(Dprpprp_smooth_t[sp],dims=['t','wprp']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())
      #Dprpprp_smooth_t_avg[sp][np.isnan(Dprpprp_smooth_t_avg[sp])]=0
      stdDprpprp_vprp_smooth.append(np.std(Dprpprp_smooth_t[sp],axis=0))

    print("species ",sp)
    print("\n ##### CHECK: integral of simulation curves (beta_i = 0.3) ##### (avg after 3.5t_A)")
    print("  -> normQ_sim = ",Qnormalization01/np.mean(np.sum(np.abs(Qprp_vprp[sp])*dvprp,axis=1)[100:]))
    print("  -> integral of df/dwprp..")
    print("     i) ..from np.gradient:",np.mean(np.sum(dfdwprp[sp]*dvprp,axis=1)[100:]))
    print("    ii) ..from finite diff:",np.mean(np.sum(dfdwprp_finitediff[sp]*dvprp,axis=1)[100:]))
    print("    ..to be compared with f(vprp=0):",np.mean(f_vprp[sp][100:,0]))
    #print("   [dfdwprp - dfdwprp_finitediff =",dfdwprp - dfdwprp_finitediff,"]")
    print("  -> integral of dQprp/dwprp: ",np.mean(np.sum(Qprp_vprp[sp]*dvprp,axis=1)[100:]))
    print("  -> integral of Dprpprp.. ")
    print("     i) ..from < dQ/dwprp > / < df/dwprp > :",np.mean(np.sum(Dprpprp[sp]*dvprp,axis=1)[100:]))
    print("    ii) ..from < (dQ/dwprp) / (df/dwprp) > :",np.mean(np.sum(Dprpprp_t_avg[sp]*dvprp,axis=1)[100:]))
    print(" ################################################################\n")

In [ ]:
plt.plot(Qprp_vprp_t_avg[0])

In [ ]:
Qprp_vprp_t_avg[0].shape

In [ ]:
Qprp_t = []
Qprp_t_avg = []
for sp in np.arange(nspecies):
    Qprp_t.append(np.sum(edotv_prp[0][:,:,:]*dvprp*dvprl,axis=(1,2)))
    Qprp_t_avg.append(xarray.DataArray(Qprp_t[sp],dims=['t']).rolling(t=rolling_window,min_periods=1, center=True).mean().to_numpy())

In [ ]:
plt.plot(Qprp_t_avg[0])

In [ ]:
#Read saved Qprps calculated from history dump dT/dt


pathToMat = "../saved-analysis/PlotQ-all.mat"
nspecies = 7
# saved .mat using 'v7.3' so need to read using hdf5
with h5py.File(pathToMat, "r") as f:
    # Print all root level object names (aka keys)
    # these can be group or dataset names
    print("Keys: %s" % f.keys())
    # get first object name/key; may or may NOT be a group
    f_group_key = list(f.keys())[1]

    # get the object type for a_group_key: usually group or dataset
    print(type(f[f_group_key]))

    # If a_group_key is a group name,
    # this gets the object names in the group and returns as a list
    data = list(f[f_group_key])

    # If a_group_key is a dataset name,
    # this gets the dataset values and returns as a list
    data = list(f[f_group_key])
    # preferred methods to get dataset values:
    #ds_obj = f[a_group_key]      # returns as a h5py dataset object
    #ds_arr = f[a_group_key][()]  # returns as a numpy array


    #data = list(f[a_group_key]['f0'])
    #ds_arr = f[a_group_key]['f0'][()]
    #t = f.get('F')['t']
    t_hst1 = f[f_group_key]['t1'][()]#[()]
    t_hst2 = f[f_group_key]['t2'][()]#[()]

    tep1 = []
    tel1 = []
    tel2 = []
    tep2 = []
    te1 = []
    te2 = []
    qp1 = []
    ql1 = []
    qp2 = []
    ql2 = []
    #hdf object reference is the key in the original file that refers to a dataset
    for i in np.arange(3):
        #[i,0]
        tep1.append(f[f[f_group_key]['tep1'][i,0]][()])
        tel1.append(f[f[f_group_key]['tel1'][i,0]][()])
        te1.append(f[f[f_group_key]['te1'][i,0]][()])
        qp1.append(f[f[f_group_key]['qp1'][i,0]][()])
        ql1.append(f[f[f_group_key]['ql1'][i,0]][()])
    for i in np.arange(nspecies):
        #[i,0]
        tep2.append(f[f[f_group_key]['tep2'][i,0]][()])
        tel2.append(f[f[f_group_key]['tel2'][i,0]][()])
        te2.append(f[f[f_group_key]['te2'][i,0]][()])
        qp2.append(f[f[f_group_key]['qp2'][i,0]][()])
        ql2.append(f[f[f_group_key]['ql2'][i,0]][()])

In [ ]:
ql2[0][0,:].shape

In [ ]:
t_hst2[0,:].shape

In [ ]:
plt.plot(qp2[1][0,:])

In [ ]:
plt.plot(t_hst2[0,:],qp2[1][0,:])
plt.plot(time,Qprp_t_avg[0])

In [ ]:
phi_t_arr = np.arange(it0,it1+1)*28.90811
dfdwprp_t_interp = []
for sp in np.arange(nspecies):
    dfdwprp_t_interp.append(np.zeros((it1+1-it0,dfdwprp_t[sp].shape[1])))
    for ind in range(dfdwprp_t[sp].shape[1]):
        dfdwprp_t_interp[sp][:,ind] = np.interp(phi_t_arr, time[:,0], dfdwprp_t[sp][:,ind])